In [1]:
import os
import time
from datetime import datetime
from typing import Dict, Any, List

from pymongo import MongoClient
import psycopg2
from psycopg2 import sql

from langchain.llms import BaseLLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
# (We’re using LLMChain for the LLM calls; the Python functions are used directly.)
from langchain_groq import ChatGroq


In [ ]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")

conn = psycopg2.connect(neon_db)
cursor = conn.cursor()

db = client["local"]

networks_collection = db["networks4"]

network_data = [
    {
        "location": "Mumbai",
        "routers": {
            "Router 1": {"status": "working", "last_checked": "2024-10-10T10:00:00Z"},
            "Router 2": {"status": "not_working", "last_checked": "2024-10-10T09:40:00Z"}
        },
        "switches": {
            "Switch 1": {"status": "not_working", "last_checked": "2024-10-10T10:05:00Z"},
            "Switch 2": {}
        },
        "repeaters": {},
        "access_points": {}
    },
    {
        "location": "Pune",
        "routers": {
            "Router 1": {"status": "working", "last_checked": "2024-10-10T10:00:00Z"},
            "Router 2": {}
        },
        "switches": {},
        "repeaters": {},
        "access_points": {}
    },{
        "location": "Aundh",
        "routers": {
            "Router 1": {"status": "working", "last_checked": "2024-10-10T10:00:00Z"},
            "Router 2": {"status": "working", "last_checked": "2024-10-10T09:40:00Z"}
        },
        "switches": {
            "Switch 1": {"status": "working", "last_checked": "2024-10-10T10:05:00Z"},
            "Switch 2": {}
        },
        "repeaters": {},
        "access_points": {}
    },
    {
        "location": "Hinjewadi",
        "routers": {
            "Router 1": {"status": "not_working", "last_checked": "2024-10-10T10:00:00Z"},
            "Router 2": {}
        },
        "switches": {},
        "repeaters": {},
        "access_points": {}
    }

]

networks_collection.insert_many(network_data)
print("Inserted documents:", list(networks_collection.find()))


In [ ]:
conn = psycopg2.connect(neondb_api)
cursor = conn.cursor()
cursor.execute("""
    CREATE TABLE IF NOT EXISTS tickets (
        ticket_id SERIAL PRIMARY KEY,
        customer_id VARCHAR(255),
        issue_type TEXT,
        location VARCHAR(255),
        status VARCHAR(50),
        created_at TIMESTAMP,
        updated_at TIMESTAMP
    );
""")
conn.commit()


In [2]:


# MongoDB Setup
client = MongoClient("mongodb://localhost:27017/")
db = client["local"]
networks_collection = db["networks4"]

# PostgreSQL Setup
conn = psycopg2.connect(neon_db)
cursor = conn.cursor()

# LLM Setup (using ChatGroq here)
api_key = os.getenv("GROQ_API_KEY", groq_api_key)
llm: BaseLLM = ChatGroq(groq_api_key=api_key, model_name="Gemma2-9b-It")


In [3]:
def networkdata_fetcher(state: Dict[str, Any]) -> Dict[str, Any]:
    print("🔍 Fetching network data from database...")
    queries: List[str] = state.get("query", [])
    results: List[str] = []
    state["location"] = ""
    state["network_status"] = "unknown"

    if not queries:
        results.append("No query provided.")
    else:
        for query in queries:
            query_words = query.lower().strip().split()
            existing_locations = networks_collection.distinct("location")
            matched_queries = [word for word in query_words if word.capitalize() in existing_locations]
            if not matched_queries:
                results.append(f"No data found for: {query}")
                continue

            regex_pattern = "|".join(matched_queries)
            matched_locations = networks_collection.find({
                "location": {"$regex": f"^{regex_pattern}$", "$options": "i"}
            })

            found = False
            for result in matched_locations:
                found = True
                state["location"] = result.get('location', "")
                network_status = result.get("status", "unknown")
                state["network_status"] = network_status.lower()
                results.append(f"Network details for {result.get('location', '')}:\n{result}")
            if not found:
                results.append(f"No data found for: {query}")

    state["response"] = "\n".join(results)
    return state


In [5]:
def ticketing_expert(state: Dict[str, Any]) -> Dict[str, Any]:
    """Create or update a ticket if a network issue is detected."""
    print("🎫 Running ticketing logic...")
    queries: List[str] = state.get("query", [])
    if not queries:
        state["response"] = "No issue details provided."
        return state

    issue_type = " ".join(queries)
    location = state.get("location", "")
    if not location:
        state["response"] = "Could not identify a valid location from the query."
        return state

    customer_id = "User-1223456543"
    current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    cursor.execute(
        sql.SQL("SELECT ticket_id, status FROM tickets WHERE customer_id = %s AND issue_type = %s AND location = %s"),
        [customer_id, issue_type, location]
    )
    existing_ticket = cursor.fetchone()

    if existing_ticket:
        ticket_id, status = existing_ticket
        cursor.execute(
            sql.SQL("UPDATE tickets SET updated_at = %s WHERE ticket_id = %s"),
            [current_datetime, ticket_id]
        )
        conn.commit()
        state["ticket_id"] = ticket_id
        state["response"] = f"Existing Ticket ID: {ticket_id} - Status: {status} - Updated At: {current_datetime}"
    else:
        cursor.execute(
            sql.SQL("INSERT INTO tickets (customer_id, issue_type, location, status, created_at) VALUES (%s, %s, %s, %s, %s) RETURNING ticket_id"),
            [customer_id, issue_type, location, "Open", current_datetime]
        )
        ticket_id = cursor.fetchone()[0]
        conn.commit()
        state["ticket_id"] = ticket_id
        state["response"] = f"New Ticket Created - ID: {ticket_id} - Status: Open - Created At: {current_datetime}"
    return state



network_expert_template = """
User Query: {query}
Network Data:
{network_data}

Based on the details above, determine whether the network is "working" or "not working". 
If any detail indicates an issue, answer "not working"; otherwise answer "working".
"""
network_expert_prompt = PromptTemplate(
    input_variables=["query", "network_data"],
    template=network_expert_template
)
network_expert_chain = LLMChain(llm=llm, prompt=network_expert_prompt, output_key="network_status")

# --- Customer Support Agent Chain ---
customer_support_template = """
User Query: {query}
Network Data:
{network_data}

Ticket Info: {ticket_info}

Chat History:
{chat_history}

Based on the information above, provide a detailed customer support response.
"""
customer_support_prompt = PromptTemplate(
    input_variables=["query", "network_data", "ticket_info", "chat_history"],
    template=customer_support_template
)
customer_support_chain = LLMChain(llm=llm, prompt=customer_support_prompt, output_key="final_response")

C:\Users\try\AppData\Local\Temp\ipykernel_16564\2288980151.py:59: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  network_expert_chain = LLMChain(llm=llm, prompt=network_expert_prompt, output_key="network_status")


In [7]:
if __name__ == "__main__":
    chat_history: List[Dict[str, Any]] = []

    print("Enter your queries (type 'exit' or 'quit' to stop):")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            break

        initial_state = {"query": [user_input], "history": chat_history, "response": ""}
        state = networkdata_fetcher(initial_state)
        network_data = state.get("response", "")
        location = state.get("location", "")
        network_expert_result = network_expert_chain.run({
            "query": user_input,
            "network_data": network_data
        })
        network_status = network_expert_result.strip().lower()
        print(f"Network Status (from LLM): {network_status}")
        if "not working" in network_status:
            state = ticketing_expert({"query": [user_input], "location": location, "response": "", "ticket_id": ""})
            ticket_info = state.get("response", "")
            ticket_id = state.get("ticket_id", "No Ticket")
        else:
            ticket_info = "No ticket required"
            ticket_id = "No Ticket"

        final_response = customer_support_chain.run({
            "query": user_input,
            "network_data": network_data,
            "ticket_info": ticket_info,
            "chat_history": "\n".join([f"User: {entry['user']}\nBot: {entry['bot']}" for entry in chat_history])
        })

        print("\nBot:\n" + final_response)

        chat_history.append({"user": user_input, "bot": final_response})

Enter your queries (type 'exit' or 'quit' to stop):
🔍 Fetching network data from database...
Network Status (from LLM): based on the provided network details, the network in pune appears to be **working**. 

here's why:

* **routers:** both router 1s listed have a status of "working". 


let me know if you have more network data to analyze!

Bot:
##  Pune Network Information 

Thank you for your request about the network in Pune. 

Based on our current records, we have information about two network deployments in Pune:

**Deployment 1:**

* **ID:** ObjectId('67a245e0222da3d790407daa')
* **Location:** Pune
* **Routers:**
    * Router 1: Status - Working, Last checked: 2024-10-10T10:00:00Z
    * Router 2:  Status - Unknown, last checked: Not available
* **Switches:** Empty
* **Repeaters:** Empty
* **Access Points:** Empty

**Deployment 2:**

* **ID:** ObjectId('67a489ab9108d9b8d778d1ad')
* **Location:** Pune
* **Routers:**
    * Router 1: Status - Working, Last checked: 2024-10-10T10:00: